### Lab 3

In [3]:
file = open("rawCorpus.txt", "r")
rawReadCorpus = file.read()
print ("Total no. of characters in read dataset: {}".format(len(rawReadCorpus)))

Total no. of characters in read dataset: 449945


In [4]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
import string

string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
string.punctuation = string.punctuation.replace('.', '')

file_new = ""
for line in rawReadCorpus:
    line_new = line.replace("\n", "")      
    file_new += line_new
preprocessedCorpus = "".join([char for char in file_new if char not in string.punctuation])

In [5]:
sentences = sent_tokenize(preprocessedCorpus)
print("1st 5 sentences of preprocessed corpus are : ")
print(sentences[0:5])
words = word_tokenize(preprocessedCorpus)
print("1st 5 words/tokens of preprocessed corpus are : ")
print(words[0:5])

1st 5 sentences of preprocessed corpus are : 
[' no it was nt black monday  but while the new york stock exchange did nt fall apart friday as the dow jones industrial average plunged N points most of it in the final hour it barely managed to stay this side of chaos  some circuit breakers installed after the october N crash failed their first test traders say unable to cool the selling panic in both stocks and futures  the N stock specialist firms on the big board floor the buyers and sellers of last resort who were criticized after the N crash once again could nt handle the selling pressure  big investment banks refused to step up to the plate to support the beleaguered floor traders by buying big blocks of stock traders say  heavy selling of standard  poor s 500stock index futures in chicago unk beat stocks downward  seven big board stocks ual amr bankamerica walt disney capital citiesabc philip morris and pacific telesis group stopped trading and never resumed  the unk has already be

In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [w for w in words if not w.lower() in stop_words]

## Creating Unigram, Bigram and Trigram Language Models

In [7]:
from collections import Counter
from nltk.util import ngrams
unigrams=[]
bigrams=[]
trigrams=[]
for content in (sentences):
    content = content.lower()
    content = word_tokenize(content)
    for word in content:
        if (word =='.'):
            content.remove(word) 
        else:
            unigrams.append(word)
    bigrams.extend(ngrams(content,2))

    trigrams.extend(ngrams(content,3))
print ("Sample of n-grams:n" + "-------------------------")
print ("--> UNIGRAMS: n" + str(unigrams[:5]) + " ...n")
print ("--> BIGRAMS: n" + str(bigrams[:5]) + " ...n")
print ("--> TRIGRAMS: n" + str(trigrams[:5]) + " ...n")

Sample of n-grams:n-------------------------
--> UNIGRAMS: n['no', 'it', 'was', 'nt', 'black'] ...n
--> BIGRAMS: n[('no', 'it'), ('it', 'was'), ('was', 'nt'), ('nt', 'black'), ('black', 'monday')] ...n
--> TRIGRAMS: n[('no', 'it', 'was'), ('it', 'was', 'nt'), ('was', 'nt', 'black'), ('nt', 'black', 'monday'), ('black', 'monday', 'but')] ...n


In [8]:
def stopwords_removal(n, a):     
    b = []
    if n == 1:
        for word in a:
            count = 0
            if word in stop_words:
                count = 0
            else:
                count = 1
            if (count==1):
                b.append(word)
        return(b)
    else:
        for pair in a:
            count = 0
            for word in pair:
                if word in stop_words:
                    count = count or 0
                else:
                    count = count or 1
            if (count==1):
                b.append(pair)
        return(b)
unigrams_Processed = stopwords_removal(1,unigrams)
bigrams_Processed = stopwords_removal(2,bigrams)
trigrams_Processed = stopwords_removal(3,trigrams)
print ("Sample of n-grams after processing:n" + "-------------------------")
print ("--> UNIGRAMS: n" + str(unigrams_Processed[:5]) + " ...n")
print ("--> BIGRAMS: n" + str(bigrams_Processed[:5]) + " ...n")
print ("--> TRIGRAMS: n" + str(trigrams_Processed[:5]) + " ...n")

Sample of n-grams after processing:n-------------------------
--> UNIGRAMS: n['nt', 'black', 'monday', 'new', 'york'] ...n
--> BIGRAMS: n[('was', 'nt'), ('nt', 'black'), ('black', 'monday'), ('monday', 'but'), ('the', 'new')] ...n
--> TRIGRAMS: n[('it', 'was', 'nt'), ('was', 'nt', 'black'), ('nt', 'black', 'monday'), ('black', 'monday', 'but'), ('monday', 'but', 'while')] ...n


In [9]:
def get_ngrams_freqDist(n, ngramList):
    ngram_freq_dict = {}
    for ngram in ngramList:
        if ngram in ngram_freq_dict:
            ngram_freq_dict[ngram] += 1
        else:
            ngram_freq_dict[ngram] = 1
    return ngram_freq_dict
    
unigrams_freqDist = get_ngrams_freqDist(1, unigrams)
unigrams_Processed_freqDist = get_ngrams_freqDist(1, unigrams_Processed)
bigrams_freqDist = get_ngrams_freqDist(2, bigrams)
bigrams_Processed_freqDist = get_ngrams_freqDist(2, bigrams_Processed)
trigrams_freqDist = get_ngrams_freqDist(3, trigrams)
trigrams_Processed_freqDist = get_ngrams_freqDist(3, trigrams_Processed)

## Predicting Next Three words using Bigram and Trigram Models


In [10]:
smoothed_bigrams_probDist = {}
V = len(unigrams_freqDist)
for i in bigrams_freqDist:
    smoothed_bigrams_probDist[i] = (bigrams_freqDist[i] + 1)/(unigrams_freqDist[i[0]]+V)
smoothed_trigrams_probDist = {}
for i in trigrams_freqDist:
    smoothed_trigrams_probDist[i] = (trigrams_freqDist[i] + 1)/(bigrams_freqDist[i[0:2]]+V)

In [11]:
testSent1 = "while friday 's debacle involved mainly professional traders rather than investors"
testSent2 = "the stock and futures markets friday will undoubtedly cause renewed debate"
testSent3 = "the sales will be used to repay its bank debt and other obligations"

In [12]:
token_1 = word_tokenize(testSent1)
token_2 = word_tokenize(testSent2)
token_3 = word_tokenize(testSent3)
ngram_1 = {1:[], 2:[]}    
ngram_2 = {1:[], 2:[]}
ngram_3 = {1:[], 2:[]}
for i in range(2):
    ngram_1[i+1] = list(ngrams(token_1, i+1))[-1]
    ngram_2[i+1] = list(ngrams(token_2, i+1))[-1]
    ngram_3[i+1] = list(ngrams(token_3, i+1))[-1]
print("Sentence 1: ", ngram_1,"Sentence 2: ",ngram_2,"Sentence 3: ",ngram_3)

Sentence 1:  {1: ('investors',), 2: ('than', 'investors')} Sentence 2:  {1: ('debate',), 2: ('renewed', 'debate')} Sentence 3:  {1: ('obligations',), 2: ('other', 'obligations')}


In [13]:
def predict_next_word(last_word,probDist):
    next_word = {}
    for k in probDist:
        if k[0] == last_word[0]:
            next_word[k[1]] = probDist[k]
    k = Counter(next_word)
    high = k.most_common(1) 
    return high[0]

def predict_next_3_words(token,probDist):
    pred1 = []
    pred2 = []
    next_word = {}
    for i in probDist:
        if i[0] == token:
            next_word[i[1]] = probDist[i]
    k = Counter(next_word)
    high = k.most_common(2)
    w1a = high[0]
    w1b = high[1]
    w2a = predict_next_word(w1a,probDist)
    w3a = predict_next_word(w2a,probDist)
    w2b = predict_next_word(w1b,probDist)
    w3b = predict_next_word(w2b,probDist)
    pred1.append(w1a)
    pred1.append(w2a)
    pred1.append(w3a)
    pred2.append(w1b)
    pred2.append(w2b)
    pred2.append(w3b)
    return pred1,pred2

print("Predicting next 3 possible word sequences with smoothed bigram model : ")
pred1,pred2 = predict_next_3_words(ngram_1[1][0], smoothed_bigrams_probDist)
print("1a)" +testSent1 +" "+ pred1[0][0]+" "+pred1[1][0]+" "+pred1[2][0])
print("1b)" +testSent1 +" "+ pred2[0][0]+" "+pred2[1][0]+" "+pred2[2][0])
pred1,pred2 = predict_next_3_words(ngram_2[1][0],smoothed_bigrams_probDist)
print("2a)" +testSent2 +" "+ pred1[0][0]+" "+pred1[1][0]+" "+pred1[2][0])
print("2b)" +testSent2 +" "+ pred2[0][0]+" "+pred2[1][0]+" "+pred2[2][0])
pred1,pred2 = predict_next_3_words(ngram_3[1][0],smoothed_bigrams_probDist)
print("3a)" +testSent3 +" "+ pred1[0][0]+" "+pred1[1][0]+" "+pred1[2][0])
print("3b)" +testSent3 +" "+ pred2[0][0]+" "+pred2[1][0]+" "+pred2[2][0])

Predicting next 3 possible word sequences with smoothed bigram model : 
1a)while friday 's debacle involved mainly professional traders rather than investors who was unk
1b)while friday 's debacle involved mainly professional traders rather than investors should be unk
2a)the stock and futures markets friday will undoubtedly cause renewed debate over the unk
2b)the stock and futures markets friday will undoubtedly cause renewed debate if the unk
3a)the sales will be used to repay its bank debt and other obligations were nt be
3b)the sales will be used to repay its bank debt and other obligations resulting from the


In [14]:
def predict_next_word(last_word,probDist):
    next_word = {}
    for k in probDist:
        if k[0:2] == last_word:
            next_word[k[2]] = probDist[k]
    k = Counter(next_word)
    high = k.most_common(1) 
    return high[0]
def predict_next_3_words(token,probDist):
    pred = []
    next_word = {}
    for i in probDist:
        if i[0:2] == token:
            next_word[i[2]] = probDist[i]
    k = Counter(next_word)
    high = k.most_common(2) 
    w1a = high[0]
    tup = (token[1],w1a[0])
    w2a = predict_next_word(tup,probDist)
    tup = (w1a[0],w2a[0])
    w3a = predict_next_word(tup,probDist)
    pred.append(w1a)
    pred.append(w2a)
    pred.append(w3a)
    return pred
print("Predicting next 3 possible word sequences with smoothed trigram model : ")
pred = predict_next_3_words(ngram_1[2],smoothed_trigrams_probDist)
print("1)" +testSent1 +" "+ pred[0][0]+" "+pred[1][0]+" "+pred[2][0])
pred = predict_next_3_words(ngram_2[2],smoothed_trigrams_probDist)
print("2)" +testSent2 +" "+ pred[0][0]+" "+pred[1][0]+" "+pred[2][0])
pred = predict_next_3_words(ngram_3[2],smoothed_trigrams_probDist)
print("3)" +testSent3 +" "+ pred[0][0]+" "+pred[1][0]+" "+pred[2][0])

Predicting next 3 possible word sequences with smoothed trigram model : 
1)while friday 's debacle involved mainly professional traders rather than investors it left the
2)the stock and futures markets friday will undoubtedly cause renewed debate about whether wall
3)the sales will be used to repay its bank debt and other obligations resulting from the
